# Image Preprocessing

In [ ]:
import keras
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

## Define The Parameters /Arguments For ImageDataGenerator Class

In [ ]:
batch_size = 32

train_datagen = ImageDataGenerator(
    shear_range=0.2,
    rotation_range=180,
    zoom_range=0.2,
    horizontal_flip=True,
)

val_datagen = ImageDataGenerator(
    rescale=1./255
)

## Applying ImageDataGenerator Functionality To Trainset And Testset

In [ ]:
train_generator = train_datagen.flow_from_directory(
    'train_set/',
    target_size=(150, 150),
    batch_size=batch_size,
    class_mode='binary'
)

val_generator = val_datagen.flow_from_directory(
    'test_set/',
    target_size=(150, 150),
    batch_size=batch_size,
    class_mode='binary'
)

Found 435 images belonging to 2 classes.
Found 121 images belonging to 2 classes.


# Model Building


### Importing Libraries

In [ ]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Activation
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import Dense

In [ ]:
model=Sequential()
model.add(Convolution2D(32,(3,3),input_shape=(150,150,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(150))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 32)      896       
_________________________________________________________________
activation (Activation)      (None, 148, 148, 32)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 175232)            0         
_________________________________________________________________
dense (Dense)                (None, 150)               26284950  
_________________________________________________________________
activation_1 (Activation)    (None, 150)               0         
_________________________________________________________________
dropout (Dropout)            (None, 150)               0

In [ ]:
model.fit(train_generator,steps_per_epoch=14,
          epochs=10,validation_data=val_generator
          ,validation_steps=4)

Epoch 1/10
14/14 [==============================] - 20s 1s/step - loss: 1035.5992 - accuracy: 0.5954 - val_loss: 0.5520 - val_accuracy: 0.8430
Epoch 2/10
14/14 [==============================] - 15s 1s/step - loss: 121.5198 - accuracy: 0.8046 - val_loss: 0.2016 - val_accuracy: 0.9174
Epoch 3/10
14/14 [==============================] - 15s 1s/step - loss: 23.5166 - accuracy: 0.8276 - val_loss: 0.2799 - val_accuracy: 0.9669
Epoch 4/10
14/14 [==============================] - 15s 1s/step - loss: 2.9886 - accuracy: 0.8552 - val_loss: 0.3461 - val_accuracy: 0.9339
Epoch 5/10
14/14 [==============================] - 15s 1s/step - loss: 0.3443 - accuracy: 0.7839 - val_loss: 0.3864 - val_accuracy: 0.9504
Epoch 6/10
14/14 [==============================] - 15s 987ms/step - loss: 1.4259 - accuracy: 0.8529 - val_loss: 0.3754 - val_accuracy: 0.9421
Epoch 7/10
14/14 [==============================] - 15s 1s/step - loss: 0.3206 - accuracy: 0.8897 - val_loss: 0.3833 - val_accuracy: 0.9504
Epoch 8/10


In [ ]:
model.save("forestfire.h5")

## CV2 Video Analysis

In [ ]:
from keras.models import load_model
from keras.preprocessing import image
import numpy as np
import cv2
from PIL import Image, ImageOps

In [ ]:
model=load_model("forest1.h5")

In [ ]:
data=np.ndarray(shape=(1,150,150,3),dtype=np.float32)
class_name=['Fire','No_Fire']
img=image.load_img('train_set/forest/NoFire (1).bmp',target_size=(64,64))
img_array = image.img_to_array(img)
img_batch = np.expand_dims(img_array, axis=0)
# x=np.expand_dims(x,axis=0)
pred=model.predict(img_batch)
index=np.argmax(pred)
class_name[index]

'Fire'

## Importing Twilio and Connecting Twilio Account 

In [ ]:
from twilio.rest import Client
from playsound import playsound

In [ ]:
model=load_model('forest1.h5')
video=cv2.VideoCapture(0)
name=['forest','with fire']

In [ ]:
account_sid='AC50d663c8a7c2d8b35b1fc09dfda93bda'
auth_token='86f345babfa094d1015a0e1137dbb679'
client =Client(account_sid,auth_token)
message=client.messages \
.create(
body='-------Forest Fire is detected,Stay Alert !!!--------',
    from_='+19457581434',to='+916369 659 356')
print(message.sid)
print("Alert Message sent")

SM66ac25dce14c53a89a912814d034dfaf
Alert Message sent


# predictions and Sending Alert Message

In [ ]:
import cv2
import numpy as np
from keras.preprocessing import image
from keras.models import load_model
from twilio.rest import Client
from playsound import playsound
model=load_model('forest1.h5')
video=cv2.VideoCapture(0)
name=['forest','with fire']
while(True):
    ret,frame=video.read()
    cv2.imshow('frame',frame)
    cv2.imwrite('image.jpg',frame)
    img=image.load_img('train_set/forest/NoFire (1).bmp',target_size=(64,64))
    x=image.img_to_array(img)
    x=np.expand_dims(x,axis=0)
    pred=model.predict(x)
    index=np.argmax(pred)
    if index==0:
        account_sid='AC50d663c8a7c2d8b35b1fc09dfda93bda'
        auth_token='86f345babfa094d1015a0e1137dbb679'
        client =Client(account_sid,auth_token)
        message=client.messages \
        .create(body='-------Fire is detected,Stay Alert !!!--------',
                from_='+19457581434',to='+916369 659 356')
        print(message.sid)
        print('Fire detected')
        print("Alert Message sent!")
        playsound('tornado-siren.mp3')
        
    else:
        print('No Danger')
        cv2.imshow("image.jpg",frame)
        if cv2.waitkey(2)&0xff == ord('q'):
            break
video.release()
cv2.destroyAllWindows()

SM200b2510df1efe889005ac80731425ff
Fire detected
Alert Message sent!
